In [1]:
import torch

# Download the MobileNetV2 model
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
# model.eval()

Using cache found in C:\Users\Amr osama abdellatif/.cache\torch\hub\pytorch_vision_v0.10.0


In [2]:
import torch.onnx

# Define dummy input for the model (1 image, 3 channels, 224x224 pixels)
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
onnx_model_path = "./output/mobilenet_v2_jit.onnx"
torch.onnx.export(model, dummy_input, onnx_model_path, verbose=True, opset_version=11)


In [7]:
import onnxruntime as ort
import numpy as np
from PIL import Image
from torchvision import transforms

# Load the ONNX model
ort_session = ort.InferenceSession("./output/mobilenet_v2_jit.onnx", providers=['CUDAExecutionProvider'])

# Preprocess the input image
def preprocess(image_path):
    # Open the image file
    image = Image.open(image_path).convert('RGB')
    
    # Define the transformation
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    # Apply the transformation
    img_tensor = preprocess(image)
    
    # Add batch dimension
    img_tensor = img_tensor.unsqueeze(0)
    
    return img_tensor.numpy()

# Perform inference
def infer(image_path):
    # Preprocess the image
    img_input = preprocess(image_path)
    
    # Run the model on the input image
    ort_inputs = {ort_session.get_inputs()[0].name: img_input}
    ort_outs = ort_session.run(None, ort_inputs)
    
    # Get the output class
    output = ort_outs[0]
    print(output.shape) # the probability of each class in the output
    predicted_class = np.argmax(output, axis=1)
    
    return predicted_class

# Provide the path to the image you want to classify
image_path = './swan.jpeg'
predicted_class = infer(image_path)
print(f'Predicted class: {predicted_class[0]}')


(1, 1000)
Predicted class: 99


c:\Users\Amr osama abdellatif\AppData\Local\Programs\Python\Python310\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [15]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
export_options = torch.onnx.ExportOptions(dynamic_shapes=True)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input,export_options=export_options)
onnx_program.save("./output/my_dynamic_model.onnx")

c:\Users\Amr osama abdellatif\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\onnx\_internal\exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(
